<a href="https://colab.research.google.com/github/Arjun-krish-10/NLP/blob/main/Streamlit_Youtube_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install streamlit

In [ ]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15985 sha256=7847a17445cedd96310f9cf578d0c4545bbc4b75bea32d6b04a6a0fb94a191b8
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
!pip install transformers 

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 47.3MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 


In [ ]:
!pip install youtube_transcript_api

In [ ]:
# from transformers import pipeline
# summary = pipeline('summarization',model='sshleifer/distilbart-cnn-12-6')


In [ ]:
import pickle


In [ ]:
%%writefile app.py

import streamlit as st
import re
import pandas as pd
import math
from youtube_transcript_api import YouTubeTranscriptApi
import time
import multiprocessing
# from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline,AutoModel
import torch


model_ids = {
    # 'Distilbart 12-9': 'sshleifer/distilbart-cnn-12-6',
    # 'Google xsum': 'google/pegasus-xsum',
    # 'Distilbart 6-6': 'sshleifer/distilbart-cnn-6-6',
    'Bart large CNN' : 'facebook/bart-large-cnn'
}

device = 0 if torch.cuda.is_available() else -1


@st.cache(allow_output_mutation=True)
def load_models():
    # return {{id: AutoModelForSeq2SeqLM.from_pretrained(mod)} if id != 'Bart large CNN' else {id:AutoModel.from_pretrained(mod)} for id,mod in model_ids.items() }
    return {id:AutoModel.from_pretrained(mod) for id,mod in model_ids.items() }

models = load_models()

@st.cache(allow_output_mutation=True, show_spinner=False)
def load_tokenizer(tok_id):
    return AutoTokenizer.from_pretrained(tok_id)

@st.cache(allow_output_mutation=True, show_spinner=False)
# def summary_fn1(nli_model_id):
def summary_fn1():
    # classifier = pipeline('summarization', model=models[nli_model_id], tokenizer=load_tokenizer(nli_model_id), device=device)
    # return pipeline('summarization', model=models[nli_model_id], tokenizer=load_tokenizer(nli_model_id), device=device)
    return pipeline('summarization', model='facebook/bart-large-cnn')#, tokenizer=load_tokenizer(nli_model_id), device=device)

# models = load_models()
classifier = summary_fn1()

def summarizer(sequence,q,i):
    outputs = classifier(sequence)
    q.put({i:outputs[0]['summary_text']})
    st.write(q)
    return #outputs[0]['summary_text']

# @st.cache
# def_load_model():
#     summary = pipeline('summarization',model='sshleifer/distilbart-cnn-12-6')
# import pandas as pd
# import re
# # pd.set_option('display.max_colwidth',1000)
# from transformers import pipeline
# import math
# # from tqdm import tqdm
# # from multiprocessing.dummy import Pool
# # pool = Pool(8)
import concurrent.futures

# def get_transcript(link):
#   try:
#     transcript = YouTubeTranscriptApi.get_transcript('link')
#   except:
#     st.except('Subtitles are disabled for this video')
def text_fn(link,row = 500,start = 0 ,end = 0):
    try:
        index = re.search('v=',link).end()
    except:
        st.error('Enter valid link')
        return
    api_seq = link[index:]
    try:
        tr = YouTubeTranscriptApi.get_transcript(api_seq)
    except:
        st.error('Subtitles not present')
        return
    df = pd.DataFrame(tr)
    # if (start != 0) & (end != 0):
    #   df
    text = ' '.join(df['text'])
    split1 = math.ceil(len(text)/row)
    
    sum_text = (text[i*row : (i+1)*row] for i in range(split1))
    return list(sum_text)
# @st.cache(suppress_st_warning=True):
# def summary_model():  
#     summary = pipeline('summarization',model='sshleifer/distilbart-cnn-12-6')

# def summary_fn1(q,text,i):
# def summary_fn1(text):
#     s = summary(text,min_length = 50)
# #   q.put({i:s[0]['summary_text']})
#     return s[0]['summary_text']

# def main():
def text_video():
    # summary = pipeline('summarization',model='sshleifer/distilbart-cnn-12-6')
    # model_desc = st.sidebar.selectbox('Model', list(model_ids.keys()), 0,key = 'mod')
    # values = st.sidebar.slider('Word range', 500,2000,key='slider')
    # st.title("Youtube summary")
    # st.subheader("Get your summary of the youtube video here")
    # link = st.text_input('Enter your youtube link',key = 'input')
    # model_id = model_ids[model_desc]

    # if st.button('Get summary'):
        # st.slider()
    text_vid = text_fn(link,row=values)
    t_vid = text_vid
    return t_vid
        # final = []
        # for i in range(len(text)):
        #     final.append(summary_fn1(model_id,text[i]))
        # sum_text = ' '.join(final)
        # st.write(sum_text)
        # starttime = time.time()
        # processes = []
        # unsorted_result = []
        # q = multiprocessing.Queue()
        # for i in range(len(text)):
            
        #     p = multiprocessing.Process(target=summary_fn1, args=(model_id,text[i],q,i))
        #     processes.append(p)
        #     p.start()
            
        # for process in processes:
        #     process.join()
        #     st.write(process)
        #     st.write(q.get())
        #     # unsorted_result.append(q.get())
        #     # q.task_done()
        #     # st.write(i)
            
        # print('Time taken = {} seconds'.format(time.time() - starttime))
        # st.write(unsorted_result)
        # st.write(len(text))
        # stime = time.time()
        # with concurrent.futures.ProcessPoolExecutor() as executor:
        #     results = executor.map(summary_fn1,text)
        # st.write(time.time() - stime)
        # for result in results:
        #     st.write(result)
# def main():
#     fin_text = text()

if __name__ == '__main__':
    model_desc = st.sidebar.selectbox('Model', list(model_ids.keys()), 0)
    values = st.sidebar.slider('Word range', 500,2000)
    st.title("Youtube summary")
    st.subheader("Get your summary of the youtube video here")
    link = st.text_input('Enter your youtube link')
    model_id = model_ids[model_desc]

    if st.button('Get summary'):
        # st.slider()
        fin_text = text_fn(link,row=values)

        fin_text = text_video()
        starttime = time.time()
        processes = []
        unsorted_result = []
        q = multiprocessing.Queue()
        for i in range(len(fin_text)):
                
            p = multiprocessing.Process(target=summarizer, args=(fin_text[i],q,i))
            processes.append(p)
            p.start()
                
        for process in processes:
            
            st.write(process)
            st.write(q.get())
            process.join()
        st.write('Time taken = {} seconds'.format(time.time() - starttime))
        # stime = time.time()
        # with concurrent.futures.ProcessPoolExecutor() as executor:
        #     results = executor.map(summary_fn1,fin_text)
        # st.write(time.time() - stime)
        # for result in results:
        #     st.write(result)
    # main()

Writing app.py


In [ ]:
# !touch app.py
# !streamlit run app.py

In [ ]:
!ngrok authtoken 1rmvOrqc2OvfRyCBtYSBaPw7roC_35VqwkJoQoowoWpmbWH31

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
# !streamlit run --server.port 80 app.py >/dev/null

In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)

In [ ]:
url

'http://737368347b7d.ngrok.io'

In [ ]:
assert False

In [ ]:
# !cat content/nohup.out

In [ ]:
ngrok.kill()